# Statistics of subjects' state

## Subject recruit

- Number of children enrolled
- Number of subjects who passed the initial screening
    - Number of subjects exited
    - Number of subjects training
    - Number of subjects who are completed experiment.
    - Number of subjects waiting to training

## Statistics of subjects who have completed the experiment

In [4]:
# Two pie plot which depict the ratio and number of adults, adolescents, children who have completed the experiment.
# one is all participants who have completed the experiment.
# another is the usable participants who have completed the experiment.

import pandas as pd
import plotly.express as px

def count_num(sub_data):
    adult_num = len(sub_data.query('Age>18'))
    adolescent_num = len(sub_data.query('(Age>12)and(Age<=18)'))
    children_num = len(sub_data.query('Age<=12'))
    return adult_num,adolescent_num,children_num

participants_tsv = r'/mnt/workdir/DCM/BIDS/participants.tsv'
participants_data = pd.read_csv(participants_tsv,sep='\t')

Number of adults: 35
Number of adolescents: 16
Number of children: 13
Adult data    Participant_ID Exp_ID          Name     Date(预约日期) Location   Age    Sex   \
0         sub_001      1  ZhouJianYang      2021.10.1       北大  23.0    male   
1         sub_002      2         XuYue      2021.10.8       北大  23.0  female   
2         sub_003      3   WangHaiTeng     2021.10.11    生物物理所  21.0    male   
4         sub_005      5     OuJianXin     2021.10.19       北大  26.0    male   
9         sub_010     Z1            郑晖     2021.11.27    生物物理所  22.0    male   
12        sub_013    Z19           王子怡      2021.12.4    生物物理所  23.0  female   
13        sub_014     Z4            邢倩      2021.12.4    生物物理所  28.0  female   
17        sub_018    Z22            李哲     2021.12.11    生物物理所  25.0  female   
20        sub_021    Z25            杨雪     2021.12.12    生物物理所  26.0  female   
22        sub_023    Z21           王之品  2021.12.14-15       北大  23.0  female   
23        sub_024     Z7            孙涛 

In [11]:
import pandas as pd
participants_tsv = r'/mnt/workdir/DCM/BIDS/participants.tsv'
participants_data = pd.read_csv(participants_tsv,sep='\t')
print(len(participants_data.query('(Age<=18)&(usable==1)')))
participants_data.query('(Age<=18)&(usable==1)')

20


,Participant_ID,Exp_ID,Name,Date(预约日期),Location,Age,Sex,Group,训练主试,dim 1,...,train_ap,train_dp,1D_ap,1D_dp,1D_acc,game1_acc,game2_train_ap,game2_train_dp,game2_test_acc,usable
10,sub_011,E11,曾典,2021.11.28,北大,11.0,male,child,王海腾,NaN,...,0.965,0.940,0.650,0.650,0.650,0.556,0.812,0.844,0.583,1.0
11,sub_012,E2,曾源,2021.11.28,北大,11.0,male,child,王海腾,NaN,...,0.865,0.890,0.642,0.758,0.700,0.512,0.656,0.812,0.516,1.0
14,sub_015,E12,邢悦萌,2021.12.5,生物物理所,12.0,female,child,陈率,NaN,...,0.960,0.895,0.983,0.983,0.983,0.683,0.812,0.719,0.714,1.0
15,sub_016,I4,王奕霏,2021.12.5,生物物理所,16.0,female,child,王海腾,NaN,...,0.820,0.900,0.850,0.933,0.892,0.714,0.750,0.844,0.798,1.0
16,sub_017,C11,胡鸣萱,2021.12.11,生物物理所,10.0,female,child,陈率,NaN,...,0.865,0.860,0.542,0.655,0.598,0.472,0.719,0.656,0.583,1.0
21,sub_022,F3,方子玥,2021.12.12,生物物理所,13.0,female,child,王海腾,NaN,...,0.885,0.885,0.750,0.708,0.729,0.667,1.000,0.938,0.821,1.0
24,sub_025,E3,李欣芮,2021.12.18,生物物理所,12.0,female,child,王海腾,NaN,...,0.810,0.820,0.558,0.642,0.600,0.560,0.812,0.906,0.536,1.0
30,sub_031,F8,孙煜伦,2021.12.25,北大,13.0,male,child,王海腾,NaN,...,0.870,0.815,0.608,0.742,0.675,0.599,0.750,0.844,0.571,1.0
31,sub_032,I3,董秋实,2021.12.25,北大,16.0,male,child,王海腾,NaN,...,0.925,0.950,0.942,0.942,0.942,0.778,0.844,0.969,0.845,1.0
34,sub_036,K7,彭子翾,2022.1.3,生物物理所,18.0,male,child,樊懿,0.925,...,0.880,0.905,0.975,0.975,0.975,0.802,0.812,0.800,0.762,1.0
